## Part 1 Complex Queries on Sakila (Multi-Table Joins)

In [2]:
Query1 = """
WITH customer_film_spend AS (
    SELECT 
        p.customer_id,
        i.film_id,
        SUM(p.amount) AS total_spent
    FROM payment p
    JOIN rental r ON p.rental_id = r.rental_id
    JOIN inventory i ON r.inventory_id = i.inventory_id
    GROUP BY p.customer_id, i.film_id
)
SELECT 
    f.title AS film_title,
    c.name AS category_name,
    cu.first_name || ' ' || cu.last_name AS customer_name,
    r.rental_date,
    cs.total_spent AS total_amount_paid
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
JOIN customer cu ON r.customer_id = cu.customer_id
JOIN customer_film_spend cs 
     ON cs.customer_id = cu.customer_id 
    AND cs.film_id = f.film_id
ORDER BY f.title, cu.last_name, r.rental_date;

"""

Query2="""
SELECT country,
       customer_name,
       total_payment
FROM (
    SELECT 
        co.country,
        cu.first_name || ' ' || cu.last_name AS customer_name,
        SUM(p.amount) AS total_payment,
        RANK() OVER (PARTITION BY co.country ORDER BY SUM(p.amount) DESC) AS rank_within_country
    FROM payment p
    JOIN rental r ON p.rental_id = r.rental_id
    JOIN customer cu ON r.customer_id = cu.customer_id
    JOIN store s ON cu.store_id = s.store_id
    JOIN address a ON s.address_id = a.address_id
    JOIN city ci ON a.city_id = ci.city_id
    JOIN country co ON ci.country_id = co.country_id
    GROUP BY co.country, cu.customer_id, cu.first_name, cu.last_name
) ranked
WHERE rank_within_country <= 3
ORDER BY country, total_payment DESC;

"""


# Convert the Query1 output to the csv file
Query3 = """
COPY (
    WITH customer_film_spend AS (
          SELECT 
              p.customer_id,
              i.film_id,
              SUM(p.amount) AS total_spent
          FROM payment p
          JOIN rental r ON p.rental_id = r.rental_id
          JOIN inventory i ON r.inventory_id = i.inventory_id
          GROUP BY p.customer_id, i.film_id
      )
      SELECT 
          f.title AS film_title,
          c.name AS category_name,
          cu.first_name || ' ' || cu.last_name AS customer_name,
          r.rental_date,
          cs.total_spent AS total_amount_paid
      FROM rental r
      JOIN inventory i ON r.inventory_id = i.inventory_id
      JOIN film f ON i.film_id = f.film_id
      JOIN film_category fc ON f.film_id = fc.film_id
      JOIN category c ON fc.category_id = c.category_id
      JOIN customer cu ON r.customer_id = cu.customer_id
      JOIN customer_film_spend cs 
           ON cs.customer_id = cu.customer_id 
          AND cs.film_id = f.film_id
      ORDER BY f.title, cu.last_name, r.rental_date
  ) TO 'customer_spend_per_movie.csv' (HEADER, DELIMITER ',');

"""

# Convert the Query2 output to the csv file
Query4 = """ 
COPY(
  SELECT country,
         customer_name,
         total_payment
  FROM (
      SELECT 
          co.country,
          cu.first_name || ' ' || cu.last_name AS customer_name,
          SUM(p.amount) AS total_payment,
          RANK() OVER (PARTITION BY co.country ORDER BY SUM(p.amount) DESC) AS rank_within_country
      FROM payment p
      JOIN rental r ON p.rental_id = r.rental_id
      JOIN customer cu ON r.customer_id = cu.customer_id
      JOIN store s ON cu.store_id = s.store_id
      JOIN address a ON s.address_id = a.address_id
      JOIN city ci ON a.city_id = ci.city_id
      JOIN country co ON ci.country_id = co.country_id
      GROUP BY co.country, cu.customer_id, cu.first_name, cu.last_name
  ) ranked
  WHERE rank_within_country <= 3
  ORDER BY country, total_payment DESC) To 'Top_customer_per_country.csv' (HEADER, DELIMITER ',');
"""



#### OUTPUT SCREENSHOT -> JOIN Challenge 


<h3>Screen Shot 1 Loading the db sakila into the duckdb</h3>
<img src="1.png">
<br>
<h3>Screen Shot 2 Testing the query</h3>
<img src="2.png">
<br>
<h3>Query 1 structure</h3>
<img src="3.png">
<br>
<h3>Output of the query results</h3>
<img src="4.png">

<br>
<br>

#### Sub Query Challenge Top Revenue Customers Per Country

<img src="5.png">



### Exported the Query OUTPUT


# PART 2 : Querying the Exported CSVs

In [3]:
# Loading the csv file and then queryiny

Query1="""
        SELECT "film_title", SUM("total_amount_paid") AS Revenue FROM 'customer_spend_per_movie.csv' GROUP BY "film_title" ORDER BY Revenue LIMIT 5 DESC;
"""

##### OUTPUT SCREENSHOT

###### Top Five Films by Revenue

<img src="7.png">

In [ ]:
# Count Unique Customer per category

Query2=""" SELECT category_name, COUNT(DISTINCT customer_name) AS customer_count FROM 'customer_spend_per_movie.csv' GROUP BY category_name ORDER BY customer_count DESC;  """

###### OUTPUT SCREENSHOT

<img src="8.png">

In [ ]:
# Find all the records for film in "Action" category where total_revenue is above the category average

query3=""" 
    SELECT
        film_title,
        category_name,
        SUM(total_amount_paid) AS revenue
    FROM 'customer_spend_per_movie.csv'
    WHERE category_name='Action'
    GROUP BY film_title, category_name
    HAVING SUM(total_amount_paid) > (
        SELECT AVG(total_amount_paid)
        FROM 'customer_spend_per_movie.csv'
        WHERE category_name='Action'
    )
    ORDER BY revenue DESC;
"""

##### OUTPUT SCREENSHOT

<img src="9.png">

<img src="10.png">

In [4]:
Query=""" WITH customer_film_revenue AS (
    SELECT 
        customer_name,
        film_title,
        SUM(total_amount_paid) AS total_revenue
    FROM 'customer_spend_per_movie.csv'
    GROUP BY customer_name, film_title
)
SELECT 
    CASE 
        WHEN total_revenue < 10 THEN '< $10'
        WHEN total_revenue BETWEEN 10 AND 50 THEN '$10 - $50'
        ELSE '> $50'
    END AS revenue_bucket,
    COUNT(*) AS pair_count
FROM customer_film_revenue
GROUP BY revenue_bucket
ORDER BY pair_count DESC;
 """

##### OUTPUT SCREENSHOT

<img src="11.png">